In [45]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

create_data_here = '/Users/anirudhkrishna/GitHub/FormulaData/csv-data/'

In [46]:
def parse_driver_name(row):
    words = row.split()
    parsed_string = ' '.join(words[:-1])
    return parsed_string

In [47]:
def FP_scrape_results(start,end,num):
    num = str(num)
    FP_results = pd.DataFrame()
    for year in range(start, end):
        races_url = f'https://www.formula1.com/en/results.html/{year}/races.html'
        response = requests.get(races_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        race_links = []
        filter_links = soup.find_all('a', attrs={'class': 'resultsarchive-filter-item-link FilterTrigger'})
        for link in filter_links:
            href = link.get('href')
            if f'/en/results.html/{year}/races/' in href:
                race_links.append(href)

        year_df = pd.DataFrame()

        round_num = 1
        for race_link in race_links:
            FP_link = race_link.replace('race-result.html', f'practice-{num}.html')
            try:
                df = pd.read_html(f'https://www.formula1.com{FP_link}')[0]
                df = df[['Pos','Driver']]
            except:
                try: 
                    ignore = pd.read_html(f'https://www.formula1.com{race_link}')[0]
                    round_num+=1
                    continue
                except:
                    continue
            df['season'] = year
            df['round'] = round_num
            df = df.loc[:, ~df.columns.str.contains('Unnamed')]
            year_df = pd.concat([year_df, df], ignore_index=True)
            round_num += 1

        FP_results = pd.concat([FP_results, year_df], ignore_index=True)
    FP_results.rename(columns={'Pos': f'fp{num}_pos'}, inplace=True)
    return FP_results

# print(FP1_results.shape)


In [48]:
FP1_results = FP_scrape_results(1984,2024,1)
FP2_results = FP_scrape_results(1988,2024,2)
FP3_results = FP_scrape_results(2003,2024,3)

In [49]:
FP1_results["Driver"] = FP1_results["Driver"].apply(parse_driver_name)
FP2_results["Driver"] = FP2_results["Driver"].apply(parse_driver_name)
FP3_results["Driver"] = FP3_results["Driver"].apply(parse_driver_name)

In [50]:
print(FP1_results.head())
print(FP2_results.head())
print(FP3_results.head())

   fp1_pos            Driver  season  round
0        1        Niki Lauda    1984      1
1        2       Alain Prost    1984      1
2        3   Elio de Angelis    1984      1
3        4  Michele Alboreto    1984      1
4        5     Nelson Piquet    1984      1
  fp2_pos            Driver  season  round
0       1      Ayrton Senna    1988      1
1       2    Gerhard Berger    1988      1
2       3   Thierry Boutsen    1988      1
3       4     Nigel Mansell    1988      1
4       5  Michele Alboreto    1988      1
   fp3_pos              Driver  season  round
0        1        Jarno Trulli    2003      1
1        2       Jenson Button    2003      1
2        3     Fernando Alonso    2003      1
3        4  Michael Schumacher    2003      1
4        5  Rubens Barrichello    2003      1


In [51]:
# FP1_data.to_csv('/Users/anirudhkrishna/GitHub/FormulaData/FP1_data.csv', index=False)

In [52]:
free_practice_results = FP1_results.merge(FP2_results, on=['Driver', 'season', 'round'], how='outer').merge(FP3_results, on=['Driver', 'season', 'round'], how='outer')


In [53]:
free_practice_results.rename(columns={'Driver':'driver_name'}, inplace=True)


In [54]:
print(free_practice_results)

       fp1_pos         driver_name  season  round fp2_pos  fp3_pos
0          1.0          Niki Lauda    1984      1     NaN      NaN
1          2.0         Alain Prost    1984      1     NaN      NaN
2          3.0     Elio de Angelis    1984      1     NaN      NaN
3          4.0    Michele Alboreto    1984      1     NaN      NaN
4          5.0       Nelson Piquet    1984      1     NaN      NaN
...        ...                 ...     ...    ...     ...      ...
16639      NaN  Narain Karthikeyan    2012     12     NaN     23.0
16640      NaN        Sergio Perez    2014     18     NaN     18.0
16641      NaN     Romain Grosjean    2015     15     NaN     13.0
16642      NaN  Antonio Giovinazzi    2017      1     NaN     20.0
16643      NaN       Robert Kubica    2021     13     NaN     19.0

[16644 rows x 6 columns]


In [55]:
free_practice_results.to_csv(create_data_here + 'free_practice_results.csv', index=False)